In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from datasets import Dataset
from tqdm import tqdm
import os

In [2]:
# 데이터 로드
test_data = pd.read_csv('./data/test.csv', encoding='utf-8-sig')

In [3]:
# 모델과 토크나이저 로드
model_dir = "./best_model_augmented"  # 체크포인트에서 모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(50358, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50358, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [7]:
# 테스트 데이터 예측
def generate_predictions(test_data, batch_size=16):
    prefix = (
        "당신은 한국어 리뷰 복원 전문가입니다.\n"
        "당신의 임무는 난독화된 한글 리뷰를 분석하고, 이를 자연스럽고 명확한 원래 의미의 한글 리뷰로 복원하는 것입니다.\n"
        "난독화된 리뷰의 단어를 원본 단어로 복원하고, 띄어쓰기와 문장 구조도 원래대로 복원하세요.\n"
        "문맥을 분석하여 자연스럽고 의미 있는 복원을 수행하며, 출력은 오직 한국어로만 작성하십시오."
    )
    
    predictions = []
    # tqdm으로 진행 상황 표시
    for i in tqdm(range(0, len(test_data), batch_size), desc="🔍 Predicting", unit="batch"):
        batch = test_data['input'][i:i + batch_size]
        inputs = [prefix + inp for inp in batch]
        
        # 토크나이징
        tokenized_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
        tokenized_inputs = {k: v.to(device) for k, v in tokenized_inputs.items()}
        
        # 모델 추론
        with torch.no_grad():
            outputs = model.generate(**tokenized_inputs, max_length=512, num_beams=5, early_stopping=True)
        
        # 결과 디코딩
        batch_predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        predictions.extend(batch_predictions)
        
        # 각 배치에서 첫 번째 샘플만 출력
        print(f"\n🔹 [Batch {i // batch_size + 1}]\n")
        print(f"📌 난독화된 입력: {batch.iloc[0]}")  # 첫 번째 샘플
        print(f"✅ 복원된 출력: {batch_predictions[0]}\n")
    
    return predictions

In [8]:
# 예측 결과 생성 및 저장
test_predictions = generate_predictions(test_data)
test_data['output'] = test_predictions
test_data.to_csv('test_results_0217.csv', encoding='utf-8-sig', index=False)

print("테스트 데이터 결과 저장 완료: test_results_0217.csv")

🔍 Predicting:   1%|▌                                                          | 1/106 [00:11<20:03, 11.46s/batch]


🔹 [Batch 1]

📌 난독화된 입력: 녀뮨넒뭅 만죡숭러윤 효템뤼에오. 푸싸눼 옰면 콕 츄쩐학꼬 싶은 콧쉰웨오. 췌꾜윕뉘댜! ㅎㅎ 당음웨 또 옭 컷 갗았요.
✅ 복원된 출력: 너무너무 만족스러운 호텔이에요. 부산에 오면 꼭 추천하고 싶은 곳이에요. 최고입니다! ᄒᄒ 다음에 또 올 것 같아요.



🔍 Predicting:   2%|█                                                          | 2/106 [00:25<22:07, 12.76s/batch]


🔹 [Batch 2]

📌 난독화된 입력: 뺨튈륨 괜찬았옅욤! 쿄테끼량 둥랐임깆또 인셔셔 좋았서온.
✅ 복원된 출력: 파티룸 괜찮았어요! 고데기랑 드라이기도 있어서 좋았어요.



🔍 Predicting:   3%|█▋                                                         | 3/106 [00:44<27:10, 15.83s/batch]

...


🔹 [Batch 104]

📌 난독화된 입력: 윈찍깠 좋꼬 팡돐 깩끝한떼 팡윽밌..ㅠㅠ
✅ 복원된 출력: 위치가 좋고 방도 깨끗한데 방음이..ᅲᅲ



🔍 Predicting:  99%|████████████████████████████████████████████████████████▍| 105/106 [22:39<00:12, 12.59s/batch]


🔹 [Batch 105]

📌 난독화된 입력: 쩡말 쵱얍귐. 윔윙쩍읊료 닿윌 켄슭쉬킵코 앓꼬댜 핑걔땍꼬. 청먈 굳탸윌롤 쟝샤햐찌 많쒜오!!!!
✅ 복원된 출력: 정말 최악임. 위치적으로 당일 취소시키고 아고다 결제당하고. 정말 그대로 장사하지 마세요!!!!



🔍 Predicting: 100%|█████████████████████████████████████████████████████████| 106/106 [22:54<00:00, 12.97s/batch]


🔹 [Batch 106]

📌 난독화된 입력: 씬냐셔 놓늦랴 방 샤친눌 체데료 못 치겉눈뗀, 널묵 깔큼학코 얀늣까꼬 멜럭있엇오! 혭확엣설 유툭 썬 굻루효뗄 봄면섧 연첸갼눈 묵건봐얀짊 했는뛔, 뜯티옇 썽췹! 찌건분틉릴 찐졀햐쉭곤 청숏토 쩔져학코 뿌뉘킥까 념뮨넋뭏 좇핥옹. 1뱍만 한 케 앝쉬웝써 타움엑 또 옮럭콩오. 탉우뭬는 확씰한 혹컁슨를 눗렬얕궤써오. 빵옘만 윗서얍췻..!! 친핥 랏윤쥠위 옳릴뚜를 륨썹픽수료 받울 숱됴 잊닦꽂 한눼오. 코충 꼬넙 방엥 쑥밗했는떼 겅찍같 졍맒 좋앗여옻!!
✅ 복원된 출력: 시내에서 오느라 방 사진을 제대로 못 찍었는데, 너무 깔끔하고 아늑하고 매력있어요! 혜화에서 글루 선 글루호텔 보면서 언젠가는 묵어봐야겠지 했는데, 드디어 성공! 직원분들이 친절하시고 청소도 철저하고 분위기가 너무너무 좋아요. 1박만 한 게 아쉬워서 다음에 또 오려고요. 다음에는 확실한 호캉스를 누리려고요. 방에만 있었으면..!! 친구 라운지의 요리를 룸서비스로 받을 수도 있다고 하네요. 고층 코너 방에 숙박했는데 경치가 정말 좋았어요!!

테스트 데이터 결과 저장 완료: test_results_0217.csv


In [9]:
import pandas as pd

# 저장된 결과 파일 경로
file_path = "test_results_0217.csv"
output_file_path = "sub_pko_aug.csv"

# 🔹 CSV 파일 로드
df = pd.read_csv(file_path, encoding="utf-8-sig")

# 🔹 'input' 컬럼 제거
df = df.drop(columns=['input'])

# 🔹 새로운 CSV 파일로 저장
df.to_csv(output_file_path, encoding="utf-8-sig", index=False)

print(f"✅ 최종 파일이 저장되었습니다: {output_file_path}")

✅ 최종 파일이 저장되었습니다: sub_pko_aug.csv
